In [ ]:
import os
import re
import sys
import glob
import pickle
from time import strftime, localtime

import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib
import matplotlib.pyplot as plt

fontsize = 8
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *

%matplotlib inline

In [ ]:
def plot_data(data_folder, H_values, var_names,
              gen_id=None, area_id=None, gen_areas_map=None,
              gen_Pnom=None, bus_id=None, add_omega_ref=True,
              xlim_distr = None, ylim=[-4,4], area_measure='momentum'):
    if gen_id is None and (bus_id is None or area_id is None):
        raise Exception('One of gen_id or (bus_id,area_id) must be not None')
    if gen_id is not None and (bus_id is not None or area_id is not None):
        raise Exception('One of gen_id or (bus_id,area_id) must be None')

    H_values = np.array(H_values)
    
    if gen_id is not None:
        inertia = {'training': H_values}
        time, x, y = load_data_generators([data_folder], [gen_id], inertia, var_names,
                                          max_block_size=500, use_tf=False)
        idx_min, _ = np.where(y['training'] == H_values.min())
        idx_max, _ = np.where(y['training'] == H_values.max())
        measure_units = 's'
    else:
        if area_measure == 'inertia':
            measure_units = 's'
            measure_abbrv = 'H'
        elif area_measure == 'energy':
            measure_units = r'GW$\cdot$s'
            measure_abbrv = 'E'
        elif area_measure == 'momentum':
            measure_units = r'GW$\cdot$s$^2$'
            measure_abbrv = 'M'
        data_files = []
        all_data_files = glob.glob(data_folder + '/*_training_set.h5')
        for data_file in all_data_files:
            _, _, areas_H, _ = read_area_values(data_file, gen_areas_map, gen_Pnom, area_measure)
            H = areas_H[area_id]
            if H in H_values:
                data_files.append(data_file)
        if len(data_files) == 0:
            raise Exception(f'There are no files with the required values of {area_measure}.')
        _, X, _ = load_data_areas({'training': all_data_files}, var_names,
                                     gen_areas_map, gen_Pnom,
                                     area_measure, max_block_size=10000,
                                     add_omega_ref = add_omega_ref, use_tf=False)
        x_train_mean = np.mean(X['training'], axis=(1,2))
        x_train_std = np.std(X['training'], axis=(1,2))
        time, x, y = load_data_areas({'training': data_files}, var_names,
                                     gen_areas_map, gen_Pnom,
                                     area_measure, max_block_size=10000,
                                     add_omega_ref = add_omega_ref, use_tf=False)
        idx_min, _ = np.where(y['training'] == H_values.min())
        idx_max, _ = np.where(y['training'] == H_values.max())
    
    for i,(m,s) in enumerate(zip(x_train_mean, x_train_std)):
        x['training'][i,:,:] = (x['training'][i,:,:] - m) / s
    N_vars = len(var_names)

    small = True
    if small:
        fig = plt.figure(figsize=(8.5 / 2.54, N_vars))
        x_offset = [0.13, 0.03]
        y_offset = [0.18, 0.03]
        x_space = 0.1
        y_space = 0.08
        rows = N_vars
        cols = 4
        w = (1 - np.sum(x_offset) - x_space * (cols - 2)) / cols
        h = (1 - np.sum(y_offset) - y_space * (rows - 1)) / rows

        ax = [[plt.axes([x_offset[0], y_offset[0] + (h + y_space) * i, 2*w, h]),
               plt.axes([x_offset[0] + 2 * w + 0.5 * x_space, y_offset[0] + (h + y_space) * i, w, h]),
               plt.axes([x_offset[0] + 3 * w + 2 * x_space, y_offset[0] + (h + y_space) * i, w, h])]
              for i in range(rows-1, -1, -1)]
        for row in ax:
            for a in row:
                for side in 'right','top':
                    a.spines[side].set_visible(False)
        cmap = [[0,0,0], [.75,.75,.75]]
        tmp_ax = []
        for j in range(3):
            for i in range(N_vars):
                tmp_ax.append(ax[i][j])
        ax = tmp_ax
    else:
        fig = plt.figure(figsize=(11, 2 * N_vars))
        gs = fig.add_gridspec(N_vars, 4)
        ax = []
        for i in range(N_vars):
            ax.append(fig.add_subplot(gs[i,:2]))
        for j in (2,3):
            for i in range(N_vars):
                ax.append(fig.add_subplot(gs[i,j]))
        cmap = [[0.2,0.8,0.2], [0.8,0.2,0.8]]

    if xlim_distr is not None:
        lim = xlim_distr
    else:
        lim = 0
    
    # variables used in FFT computation
    block_size = 5
    N_samples = block_size * x['training'].shape[-1]
    dt = np.diff(time[:2])[0]
    freq = fftfreq(N_samples, dt)[:N_samples//2]

    for i in range(N_vars):
        ### actual traces
        ax[i].plot(time, x['training'][i, idx_min[:5], :].T, color=cmap[0], lw=0.75)
        ax[i].plot(time, x['training'][i, idx_max[:5], :].T, color=cmap[1], lw=0.75)
        ax[i].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
        ax[i].set_xticks(np.r_[0 : 61 : 20])
        ax[i].set_yticks(np.r_[-4 : 4.1 : 2])
        if i != N_vars - 1:
            ax[i].set_xticklabels([])
        ### distributions
        n,edges = np.histogram(np.ndarray.flatten(x['training'][i,:,:][idx_min,:]), \
                               bins=100, range=ylim, density=True)
        lim = n.max() if n.max() > lim else lim
        ax[i+N_vars].plot(n, edges[:-1], color=cmap[0], linewidth=1.2)
        n,edges = np.histogram(np.ndarray.flatten(x['training'][i,:,:][idx_max,:]), \
                               bins=100, range=ylim, density=True)
        lim = n.max() if n.max() > lim else lim
        ax[i+N_vars].plot(n, edges[:-1], color=cmap[1], linewidth=1.2)
        ax[i+N_vars].set_yticklabels([])
        ax[i+N_vars].set_xticks([0, 0.5])
        ax[i+N_vars].set_yticks(np.r_[-4 : 4.1 : 2])
        if i != N_vars - 1:
            ax[i+N_vars].set_xticklabels([])
        ax[i+N_vars].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
        
        ### FFTs
        N_traces = idx_min.size // block_size
        Xf = np.zeros((N_traces, N_samples), dtype=np.complex64)
        for j in range(N_traces):
            start_trace = j * block_size
            X = np.vstack(x['training'][i, idx_min[start_trace : start_trace + block_size], :]).flatten()
            Xf[j,:] = fft(X)
        Xf = 2.0 / N_samples * np.abs(Xf[:, 0 : N_samples // 2])
        ax[i+2*N_vars].plot(freq, 20*np.log10(Xf.mean(axis=0)), color=cmap[0], lw=1, \
                                label=f'{measure_abbrv}={H_values[0]:.3f} {measure_units}')
        N_traces = idx_max.size // block_size
        Xf = np.zeros((N_traces, N_samples), dtype=np.complex64)
        for j in range(N_traces):
            start_trace = j * block_size
            X = np.vstack(x['training'][i, idx_max[start_trace : start_trace + block_size], :]).flatten()
            Xf[j,:] = fft(X)
        Xf = 2.0 / N_samples * np.abs(Xf[:, 0 : N_samples // 2])
        ax[i+2*N_vars].plot(freq, 20*np.log10(Xf.mean(axis=0)), color=cmap[1], lw=1, \
                                label=f'{measure_abbrv}={H_values[1]:.3f} {measure_units}')
        if i != N_vars - 1:
            ax[i+2*N_vars].set_xticklabels([])
       
    for a in ax:
        for side in 'right', 'top':
            a.spines[side].set_visible(False)
    for i,var_name in enumerate(var_names):
        ax[i].set_ylabel('Norm. ' + var_name)

    for i in range(N_vars):
        ax[i].set_ylim(ylim)
        ax[N_vars + 1].set_ylim(ylim)
        ax[N_vars + i].plot([0, lim*1.05], [0,0], '--', lw=1, color=[.6,.6,.6])
        ax[2*N_vars + i].set_ylabel('Power [dB]')
    ax[-1].set_xlabel('Frequency [Hz]')
    
    ax[N_vars - 1].set_xlabel('Time [s]')
    ax[2 * N_vars - 1].set_xlabel('Fraction')
    if not small:
        ax[3*N_vars-1].legend(loc='upper right', frameon=False)
        fig.tight_layout(pad=0.1)
    if gen_id is not None:
        fig.savefig(data_folder + f'/generator_{gen_id}.pdf');
    else:
        fig.savefig(data_folder + f'/bus_{bus_id}.pdf');

## Two area network

## IEEE39

In [ ]:
bus_IDs = 3,# 14, 17, 39
area = 1
H_G1 = 500
data_folder = f'../data/IEEE39/var_H_area_{area}/IEEE39_D=2_DZA=0.0/H_G1_500/stoch_load_bus_3'
gen_areas_map = [['Pg31','Pg32','Pg39'], ['Pg33','Pg34','Pg35','Pg36'], ['Pg37','Pg38'], ['Pg30']]
gen_Pnom = {gen: 100e6 for area in gen_areas_map for gen in area}
area_measure = 'momentum'
print(' ' * 58 + '{:^14s}'.format(area_measure.upper()))
print('=' * 77)
print('{:^56s} {:^6s} {:^6s} {:^6s}'.format('Filename', 'A1','A2','A3'))
print('=' * 77)
for data_file in glob.glob(data_folder + '/*_training_set.h5'):
    gen_IDs, gen_H, area_H, _ = read_area_values(data_file, gen_areas_map, gen_Pnom, area_measure)
    a,b = os.path.basename(data_file)[:30], os.path.basename(data_file)[-23:]
    print(f'{a}...{b} {area_H[0]:6.3f} {area_H[1]:6.3f} {area_H[2]:6.3f}')

In [ ]:
if area_measure == 'momentum':
    if area in (1,2):
        H_values = [8/30, 14/30]
    elif area == 3:
        H_values = [0.1, 0.3]
# var_names_fmt = 'omegael_bus{}','Pe_bus{}','Qe_bus{}','Vd_bus{}','Vq_bus{}'
# var_names_fmt = 'Pe_bus{}','Qe_bus{}','Vd_bus{}','Vq_bus{}'
var_names_fmt = 'Vd_bus{}', 'Pe_bus{}'
for bus in bus_IDs:
    var_names = tuple(var_name.format(bus) for var_name in var_names_fmt)
    plot_data(data_folder, H_values, var_names, bus_id=bus, area_id=area-1,
              gen_areas_map=gen_areas_map, gen_Pnom=gen_Pnom, xlim_distr = 0.5, ylim=[-4,4])

## IEEE39 with DIgSILENT PowerFactory

In [ ]:
bus_IDs = 3,
area = 1
data_folder = f'../data/IEEE39/PowerFactory/var_H_area_{area}'
gen_areas_map = [['G 02','G 03','G 10'], ['G 04','G 05','G 06','G 07'], ['G 08','G 09'], ['G 01']]
gen_Pnom = {'G 01': 10000e6, 'G 02': 700e6, 'G 03': 800e6, 'G 04': 800e6, 'G 05': 300e6,
            'G 06': 800e6, 'G 07': 700e6, 'G 08': 700e6, 'G 09': 1000e6, 'G 10': 1000e6}
area_measure = 'inertia'
print(' ' * 58 + '{:^14s}'.format(area_measure.upper()))
print('=' * 77)
print('{:^56s} {:^6s} {:^6s} {:^6s}'.format('Filename', 'A1','A2','A3'))
print('=' * 77)
for data_file in glob.glob(data_folder + '/*_training_set.h5'):
    gen_IDs, gen_H, area_H, _ = read_area_values(data_file, gen_areas_map, gen_Pnom, area_measure)
    a,b = os.path.basename(data_file)[:30], os.path.basename(data_file)[-23:]
    print(f'{a}...{b} {area_H[0]:6.3f} {area_H[1]:6.3f} {area_H[2]:6.3f}')

In [ ]:
H_values = [3.00004, 5.1002]
# var_names_fmt = 'omegael_bus{}','Pe_bus{}','Qe_bus{}','Vd_bus{}','Vq_bus{}'
var_names_fmt = 'Pe_bus{}','Qe_bus{}','Vd_bus{}','Vq_bus{}'
for bus in bus_IDs:
    var_names = tuple(var_name.format(bus) for var_name in var_names_fmt)
    plot_data(data_folder, H_values, var_names, bus_id=bus, area_id=area-1,
              gen_areas_map=gen_areas_map, gen_Pnom=gen_Pnom, xlim_distr = 0.5, ylim=[-4,4],
              area_measure='inertia', add_omega_ref=False)